In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [2]:
from google.colab import files
files.upload()

#Potrebno je uploadati text s kojim radimo

Saving seinfield.txt to seinfield (1).txt


{'seinfield.txt': b'jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! you wanna go to sleep, you wanna get up, you wanna go out again tomorrow, right? where ever you are in life, its my feeling, youve gotta 

In [3]:
file = open('/content/seinfield.txt', 'r')

In [4]:
text = file.read()

In [5]:
len(text)

#Preko 3 milijuna charactera skripte

3342325

In [6]:
text[:1000]

'jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! you wanna go to sleep, you wanna get up, you wanna go out again tomorrow, right? where ever you are in life, its my feeling, youve gotta go. \n\njerry: (poi

In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print(len(vocab), 'unique characters')

72 unique characters


Vektorizacija

In [8]:
primjer = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(primjer, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [9]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

#StringLookup layer će pretvarati tokene u ID-eve (brojeve)

In [10]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[39, 40, 41, 42, 43, 44, 45], [62, 63, 64]]>

Pošto je cilj generiranje teksta, biti će važno obrnuti ovu reprezentaciju i oporaviti stringove koje možemo čitati. Za ovo se koristi preprocessing.StringLookup(..., invert=True).

In [11]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [12]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [13]:
tf.strings.reduce_join(chars, axis=-1).numpy()

#Spajamo charactere natrag u stringove

array([b'abcdefg', b'xyz'], dtype=object)

In [14]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

**Zadatak**: S obzirom na prošli character, koji će biti najvjerojatniji idući character?

**Stvaranje primjera za trening**: Mi samo imamo naš text file, i koristeći njega trebamo stvoriti vlastite podatke za treniranje i testiranje.

Podijeliti ćemo tekst na primjere sekvenci znakova. Svaka unosna sekvenca će imati svoju duljinu, za koju je cilj ista ta sekvenca pomaknuta za jedno polje udesno. Npr. za Seinfeld:
Input: Seinfiel
Željeni output: einfield


In [15]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

#Prije smo imali tekst za primjer, ovdje koristimo naš pravi tekst

<tf.Tensor: shape=(3342325,), dtype=int64, numpy=array([48, 43, 56, ..., 47, 52, 45])>

In [16]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [17]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

j
e
r
r
y
:
 
d
o
 


In [18]:
seq_length = 100  #Znači da će svaki primjer koristiti 100 znakova odjednom za treniranje
examples_per_epoch = len(text)//(seq_length+1)

In [19]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
#seq + 1 zbog pomaka udesno
#batch metodom se koristimo da znakove pretvorimo u sekvence lakše

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'j' b'e' b'r' b'r' b'y' b':' b' ' b'd' b'o' b' ' b'y' b'o' b'u' b' '
 b'k' b'n' b'o' b'w' b' ' b'w' b'h' b'a' b't' b' ' b't' b'h' b'i' b's'
 b' ' b'i' b's' b' ' b'a' b'l' b'l' b' ' b'a' b'b' b'o' b'u' b't' b'?'
 b' ' b'd' b'o' b' ' b'y' b'o' b'u' b' ' b'k' b'n' b'o' b'w' b',' b' '
 b'w' b'h' b'y' b' ' b'w' b'e' b'r' b'e' b' ' b'h' b'e' b'r' b'e' b'?'
 b' ' b't' b'o' b' ' b'b' b'e' b' ' b'o' b'u' b't' b',' b' ' b't' b'h'
 b'i' b's' b' ' b'i' b's' b' ' b'o' b'u' b't' b'.' b'.' b'.' b'a' b'n'
 b'd' b' ' b'o'], shape=(101,), dtype=string)


In [20]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and o'
b'ut is one of the single most enjoyable experiences of life. people...did you ever hear people talking'
b' about we should go out? this is what theyre talking about...this whole thing, were all out now, no o'
b'ne is home. not one person here is home, were all out! there are people trying to find us, they dont '
b'know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt '


In [21]:
#Napokon, funkcija koja uzima sekvencu kao input, udvostruči je i shifta kopiju udesno
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [22]:
split_input_target(list("Seinfield")) #primjer od ranije

(['S', 'e', 'i', 'n', 'f', 'i', 'e', 'l'],
 ['e', 'i', 'n', 'f', 'i', 'e', 'l', 'd'])

In [43]:
dataset = sequences.map(split_input_target)

In [44]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and '
Target: b'erry: do you know what this is all about? do you know, why were here? to be out, this is out...and o'


Iskoristili smo tf.data za razdvojiti tekst u sekvence s kojima možemo raditi. Ali prije pohranjivanja u model, moramo podatke shuffle-ati i zapakirati ih u batcheve.

In [45]:
# Batch size
BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 30000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((128, 100), (128, 100)), types: (tf.int64, tf.int64)>

**MODEL**

In [46]:
# Length of the vocabulary in chars
vocab_size=len(ids_from_chars.get_vocabulary())
#vocab_size=100

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import SimpleRNN, LSTM
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

In [57]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)
    #self.dense2 = tf.keras.layers.Dense(40)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)
    #x = self.dense2(x, training = training)

    if return_state:
      return x, states
    else:
      return x

In [58]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [59]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 100, 73) # (batch_size, sequence_length, vocab_size)


In [60]:
model.summary()

Model: "my_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  18688     
_________________________________________________________________
gru_2 (GRU)                  multiple                  3938304   
_________________________________________________________________
dense_4 (Dense)              multiple                  74825     
Total params: 4,031,817
Trainable params: 4,031,817
Non-trainable params: 0
_________________________________________________________________


In [61]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [62]:
sampled_indices

array([49, 21, 42, 39, 59, 52, 28, 58, 42, 10,  2, 61, 17,  3, 40,  7, 48,
       14, 20, 71, 37, 14, 48, 43, 20, 36, 50, 16,  2, 33, 11, 59, 17, 65,
       23, 14, 23, 24, 70, 12, 43, 61, 59, 58, 48, 72, 32, 67, 23, 51, 66,
       12, 29, 47, 53, 24, 39, 35, 26, 61, 49, 15, 44, 15, 67, 43, 37, 43,
        2, 42, 31, 65,  4,  8, 57, 41, 19, 66, 41, 11,  6, 43, 30, 21, 63,
       39,  8,  8, 16, 25, 52, 25, 16, 54, 11, 19,  0,  0, 11, 69])

In [63]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

#Predviđanja netreniranog modela. Nadam se da će nakon treninga biti bolji.

Input:
 b'where we are? 30 000 feet above your head, you bastard! (the 3 laugh and howl) \n\ngeorge: what are th'

Next Char Predictions:
 b'k3daun:td( w/!b%j,2\xc2\x94_,je2]l. ?)u/{5,56\xc2\x93*ewutj\xe2\x80\xa6>~5m}*;io6a\\8wk-f-~e_e d={"&sc1}c)$e<3ya&&.7n7.p)1[UNK][UNK])\xc2\x92'


In [64]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [65]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (128, 100, 73)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.28942


In [66]:
model.compile(optimizer='adam', loss=loss)

In [67]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [68]:
EPOCHS = 20
    

In [70]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
258/258 [==============================] - 60s 216ms/step - loss: 0.7829
Epoch 2/20
258/258 [==============================] - 60s 214ms/step - loss: 0.7700
Epoch 3/20
258/258 [==============================] - 60s 215ms/step - loss: 0.7589
Epoch 4/20
258/258 [==============================] - 60s 214ms/step - loss: 0.7483
Epoch 5/20
258/258 [==============================] - 60s 216ms/step - loss: 0.7398
Epoch 6/20
258/258 [==============================] - 60s 215ms/step - loss: 0.7328
Epoch 7/20
258/258 [==============================] - 60s 215ms/step - loss: 0.7266
Epoch 8/20
258/258 [==============================] - 60s 215ms/step - loss: 0.7211
Epoch 9/20
258/258 [==============================] - 60s 214ms/step - loss: 0.7159
Epoch 10/20
258/258 [==============================] - 60s 215ms/step - loss: 0.7119
Epoch 11/20
258/258 [==============================] - 60s 214ms/step - loss: 0.7081
Epoch 12/20
258/258 [==============================] - 60s 216ms/step - lo

In [71]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [72]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [1]:
start = time.time()
states = None
next_char = tf.constant(['jerry:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

NameError: ignored